# Convert files to netcdf format
Convert yearly emission inventory to *.netcdf format.<br>
Conversion is based on original script from Lionel Constantin from EMPA.<br>

In [1]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from emiproc.inventories import Inventory
from emiproc.grids import RegularGrid
from emiproc.regrid import remap_inventory
from emiproc.plots import plot_inventory
from emiproc.exports.rasters import export_raster_netcdf
from emiproc.exports.netcdf import nc_cf_attributes
from emiproc.utilities import Units
from emiproc.inventories.utils import drop


plt.style.use("default")

### Load inventory and related files

In [34]:
# load inventory gpkg
YEAR = 2024

root = os.path.dirname(os.path.abspath(''))
path = os.path.join(root, 'output', str(YEAR), f'Munich_GHGco_v2_2_year{YEAR}.gpkg')
gdf = gpd.read_file(path)

#gdf.geometry = gdf.geometry.centroid # convert polygons to points

gdf.head()

,id,ISO3,Year,GNFR_Sector,SourceType,CO2,NOX,CO,CH4,PM,CO2_bio,geometry
0,97.0,DEU,2024,D,A,111.620497,0.000000,0.000000,0.728119,NaN,NaN,"POLYGON ((675503.47 5336499.375, 675500.627 53..."
1,97.0,DEU,2024,HR_outdoor,A,47.942039,NaN,NaN,NaN,NaN,47.942039,"POLYGON ((675503.47 5336499.375, 675500.627 53..."
2,97.0,DEU,2024,F_hot,A,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,"POLYGON ((675503.47 5336499.375, 675500.627 53..."
3,97.0,DEU,2024,K,A,0.000000,0.000000,0.000000,546.161562,NaN,NaN,"POLYGON ((675503.47 5336499.375, 675500.627 53..."
4,97.0,DEU,2024,J,A,0.000000,0.036967,0.720756,0.003186,NaN,NaN,"POLYGON ((675503.47 5336499.375, 675500.627 53..."


In [35]:
# Instantiate emiproc inventory
substances = ['CO2', 'NOX', 'CO', 'CH4', 'PM', 'CO2_bio']
sectors = gdf['GNFR_Sector'].unique()

inv = Inventory.from_gdf(gdfs={
    category: gdf.loc[gdf['GNFR_Sector'] == category, substances + ['geometry']] for category in sectors
},
    name='Munich',
)

# create regular grid based on TNO grid bounds
TNO_grid = gpd.read_file(os.path.join(root, 'data', 'TNO_100m_grid_munich.gpkg'))
x_min, y_min, x_max, y_max = TNO_grid.total_bounds

grid_tno = RegularGrid(
    xmin=x_min, ymin=y_min, xmax=x_max, ymax=y_max, dx=1/600, dy=1/1200)

grid_footprint = RegularGrid(
    xmin=np.floor(x_min*100)/100, ymin=np.floor(y_min*100)/100,
    xmax=np.ceil(x_max*100)/100, ymax=np.ceil(y_max*100)/100+0.01, dx=0.01, dy=0.01)

# remap inventor to regular Grid
inv_mapped_100m = remap_inventory(inv, grid_tno)
inv_mapped_1km = remap_inventory(inv, grid_footprint)

inv_mapped_100m.year = YEAR
inv_mapped_100m.name = "Munich GHG & co-emitted species inventory"

inv_mapped_1km.year = YEAR
inv_mapped_1km.name = "Munich GHG & co-emitted species inventory"

In [36]:
#plot_inventory(inv_mapped_1km)

In [37]:
filepath = export_raster_netcdf(
    inv=inv_mapped_1km,
    path=os.path.join(root, 'output', str(YEAR), f'Munich_GHGco_v2_2_year{YEAR}_FP_1km_grid.nc'),
    group_categories=True,
    unit=Units.KG_PER_YEAR,
    var_name_format="{substance}",
    lon_name="lon",
    lat_name="lat",
    netcdf_attributes=nc_cf_attributes(
        references="https://github.com/tum-esm/tum-mci",
        institution="Technical University Munich (TUM)",
        author="Daniel Kühbacher, TUM",
        contact="daniel.kuehbacher@tum.de",
        title=f"Munich City Emissions Inventory {YEAR}",
        source="Bottom-Up calculations based on local data for GNFR A, C, F and Human Respiration. National downscaled for all other sectors (TNO-GHGco v4.1)",
        comment=("The data was generated by a team from the Technical University Munich (TUM) and TNO."),
        additional_attributes={
            "TUM_Authors": [
                "Kühbacher, Daniel <daniel.kuehbacher@tum.de>",
                "Aigner, Patrick <patrick.aigner@tum.de>",
                "Chen, Jia <jia.chen@tum.de>",
                "Hinderer, Julian <julian.hinderer@tum.de>",
                "Suhendra, Michael <michael.suhendra@tum.de>",
                "Yirtar, Beyza <beyza.yirtar@tum.de>"
            ],
            "TNO_Authors": [
                "Super, Ingrid, <ingrid.super@tno.nl>",
                "Denier van der Gon, Hugo <hugo.deniervandergon@tno.nl>",
                "Dröge, Rianne <rianne.droge@tno.nl>",
                "Schoenecker, Emma <emma.schoenmakers@tno.nl>",
                "Hohenberger, Tilman <tilman.hohenberger@tno.nl>"
            ]
        }
    ),
)

In [38]:
filepath = export_raster_netcdf(
    inv=inv_mapped_100m,
    path=os.path.join(root, 'output', str(YEAR), f'Munich_GHGco_v2_2_year{YEAR}_TNO_100m_grid.nc'),
    group_categories=True,
    unit=Units.KG_PER_YEAR,
    var_name_format="{substance}",
    lon_name="lon",
    lat_name="lat",
    netcdf_attributes=nc_cf_attributes(
        references="https://github.com/tum-esm/tum-mci",
        institution="Technical University Munich (TUM)",
        author="Daniel Kühbacher, TUM",
        contact="daniel.kuehbacher@tum.de",
        title=f"Munich City Emissions Inventory {YEAR}",
        source="Bottom-Up calculations based on local data for GNFR A, C, F and Human Respiration. National downscaled for all other sectors (TNO-GHGco v4.1)",
        comment=("The data was generated by a team from the Technical University Munich (TUM) and TNO."),
        additional_attributes={
            "TUM_Authors": [
                "Kühbacher, Daniel <daniel.kuehbacher@tum.de>",
                "Aigner, Patrick <patrick.aigner@tum.de>",
                "Chen, Jia <jia.chen@tum.de>",
                "Hinderer, Julian <julian.hinderer@tum.de>",
                "Suhendra, Michael <michael.suhendra@tum.de>",
                "Yirtar, Beyza <beyza.yirtar@tum.de>"
            ],
            "TNO_Authors": [
                "Super, Ingrid, <ingrid.super@tno.nl>",
                "Denier van der Gon, Hugo <hugo.deniervandergon@tno.nl>",
                "Dröge, Rianne <rianne.droge@tno.nl>",
                "Schoenecker, Emma <emma.schoenmakers@tno.nl>",
                "Hohenberger, Tilman <tilman.hohenberger@tno.nl>"
            ]
        }
    ),
)

In [39]:
ds = xr.open_dataset(os.path.join(root, 'output', str(YEAR), f'Munich_GHGco_v2_2_year{YEAR}_FP_1km_grid.nc'))
ds

<xarray.Dataset> Size: 599kB
Dimensions:                  (category: 14, lat: 21, lon: 39, substance: 6,
                              lon_bnds: 40, lat_bnds: 22, time: 1)
Coordinates:
  * substance                (substance) <U7 168B 'CO2_bio' 'CO2' ... 'NOX' 'CO'
  * category                 (category) <U17 952B 'A' 'HR_residential' ... 'I'
  * lon                      (lon) float64 312B 11.36 11.37 ... 11.72 11.74
  * lat                      (lat) float64 168B 48.05 48.06 ... 48.24 48.26
  * lon_bnds                 (lon_bnds) float64 320B 11.35 11.36 ... 11.73 11.74
  * lat_bnds                 (lat_bnds) float64 176B 48.05 48.06 ... 48.25 48.26
  * time                     (time) datetime64[ns] 8B 2024-07-01
Data variables: (12/19)
    CO2_bio                  (category, lat, lon) float64 92kB ...
    CO2                      (category, lat, lon) float64 92kB ...
    CH4                      (category, lat, lon) float64 92kB ...
    PM                       (category, lat, lon) float64 92kB ...
    NOX                      (category, lat, lon) float64 92kB ...
    CO                       (category, lat, lon) float64 92kB ...
    ...                       ...
    emi_PM_total             (category) float64 112B ...
    emi_NOX_all_sectors      (lat, lon) float64 7kB ...
    emi_NOX_total            (category) float64 112B ...
    emi_CO_all_sectors       (lat, lon) float64 7kB ...
    emi_CO_total             (category) float64 112B ...
    cell_area                (lat, lon) float64 7kB ...
Attributes: (12/14)
    Conventions:      CF-1.10
    title:            Munich City Emissions Inventory 2024
    comment:          The data was generated by a team from the Technical Uni...
    source:           Bottom-Up calculations based on local data for GNFR A, ...
    history:          2025-09-04 12:35:36: created by emiproc ;\n
    references:       https://github.com/tum-esm/tum-mci
    ...               ...
    contact:          daniel.kuehbacher@tum.de
    creation_time:    2025-09-04 12:35:36
    TUM_Authors:      ['Kühbacher, Daniel <daniel.kuehbacher@tum.de>', 'Aigne...
    TNO_Authors:      ['Super, Ingrid, <ingrid.super@tno.nl>', 'Denier van de...
    emiproc_history:  ["Inventory(Inventory) created as type:'Inventory'", 'C...
    year:             2024